In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm

IMAGE_FOLDER = "Helmet_Detection/images"
ANNOTATION_FOLDER = "Helmet_Detection/annotations"

OUTPUT_FOLDER = "dataset_cropped"
WITH_HELMET = os.path.join(OUTPUT_FOLDER, "with_helmet")
WITHOUT_HELMET = os.path.join(OUTPUT_FOLDER, "without_helmet")

os.makedirs(WITH_HELMET, exist_ok=True)
os.makedirs(WITHOUT_HELMET, exist_ok=True)

def crop_and_save(image_path, label, box, count):
    img = cv2.imread(image_path)
    x_min, y_min, x_max, y_max = map(int, box)
    cropped = img[y_min:y_max, x_min:x_max]

    if cropped.size == 0:
        return  

    output_dir = WITH_HELMET if label == 'helmet' else WITHOUT_HELMET
    save_path = os.path.join(output_dir, f"{label}_{count}.jpg")
    cv2.imwrite(save_path, cropped)

count = 0
for xml_file in tqdm(os.listdir(ANNOTATION_FOLDER)):
    if not xml_file.endswith(".xml"):
        continue

    tree = ET.parse(os.path.join(ANNOTATION_FOLDER, xml_file))
    root = tree.getroot()

    image_file = root.find('filename').text
    image_path = os.path.join(IMAGE_FOLDER, image_file)

    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in ['helmet', 'head']:  
            continue

        bndbox = obj.find('bndbox')
        box = [
            bndbox.find('xmin').text,
            bndbox.find('ymin').text,
            bndbox.find('xmax').text,
            bndbox.find('ymax').text
        ]
        crop_and_save(image_path, label if label == 'helmet' else 'no_helmet', box, count)
        count += 1

print("Selesai cropping dan menyimpan gambar ke folder klasifikasi.")


100%|██████████| 764/764 [00:00<00:00, 1545.99it/s]

Selesai cropping dan menyimpan gambar ke folder klasifikasi.
